In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex as re

from numpy import array
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, Conv1D
from keras.preprocessing.text import Tokenizer

In [3]:
data = pd.read_csv("website_classification.csv")
df = (data [["cleaned_website_text", "Category"]]).dropna(axis=0)
df

,cleaned_website_text,Category
0,official site good hotel accommodation big sav...,Travel
1,expedia hotel book sites like use vacation wor...,Travel
2,tripadvisor hotel book sites like previously d...,Travel
3,cheap flights search compare flights momondo f...,Travel
4,bot create free account create free account si...,Travel
...,...,...
1403,old nude women porn mature granny sex horny ol...,Adult
1404,bdsm cams bdsm chat bondage cams free bdsm vid...,Adult
1405,porno dvd online european porn dvd cheap adult...,Adult
1406,anal dream house anal dream house anal dream h...,Adult


In [4]:
def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [6]:
X = []
sentences = list(df["cleaned_website_text"])
for sen in sentences:
    X.append(preprocess_text(sen))

In [11]:
def process (x):
    if x =="Travel": return 1
    elif x=="Social Networking and Messaging": return 2
    elif "News": return 3
    elif "Streaming Services": return 4
    elif "Sports": return 5
    elif "Photography": return 6
    elif "Law and Government": return 7
    elif "Health and Fitness": return 8
    elif "Games": return 9
    elif "Forums": return 10
    elif "Food": return 11
    elif "Education": return 12
    elif "E-Commerce": return 13
    elif "Computers and Technology": return 14
    elif "Business/Corporate": return 15
    else: return 0


y = np.array(list(map(process, df["Category"])))
y

array([1, 1, 1, ..., 3, 3, 3])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [13]:
##hyper parameters
batch_size = 32
embedding_dims = 75 #Length of the token vectors
filters = 250 #number of filters in your Convnet
kernel_size = 3 # a window size of 3 tokens
hidden_dims = 250 #number of neurons at the normal feedforward NN
epochs = 2
maxlen=100

In [14]:
model = Sequential()
# model.add(Conv1D(filters,kernel_size,padding = 'valid' , activation = 'relu',strides = 1 ))
# model.add(GlobalMaxPooling1D())
#GlobalMaxPooling1D(n) default = 2.
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [15]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train,y_train,batch_size = batch_size,epochs = epochs , validation_data = (X_test,y_test))

Epoch 1/2
36/36 [==============================] - 1s 10ms/step - loss: -30506.8926 - accuracy: 0.0737 - val_loss: -79580.4688 - val_accuracy: 0.0674
Epoch 2/2
36/36 [==============================] - 0s 3ms/step - loss: -163863.6406 - accuracy: 0.0782 - val_loss: -266583.2500 - val_accuracy: 0.0674


In [16]:
pred = model.predict(X_test)
pred = np.array(list(map(lambda x : int(x[0]), pred)))
accuracy = accuracy_score(pred,y_test)
accuracy

9/9 [==============================] - 0s 1ms/step


0.0673758865248227